# **<font color = '#3498eb'>OBJETIVES</font>**

Hi 

# **<font color = '#3498eb'>SECTOR, COMPANY</font>**

# **<font color = '#3498eb'>SOME QUESTIONS TO START THE ANALYSIS</font>**

# **<font color = '#3498eb'>LIBRARIES</font>**

In [3]:
pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import sqlalchemy as sa

# Mejorar legibilidad
import tabulate

# **<font color = '#3498eb'>DATA LOADING AND UNDERSTANDING</font>**

# **<font color = '#3498eb'>DATA QUALITY</font>**